## An Example Analysing NICER Data One Sciserver

In this tutorial, we will go through the steps of analyzing a NICER observation of `PSR_B0833-45` (`obsid = 4142010107`) using `heasoftpy`.

The following assumes this notebook is run from the (heasoft) environment on Sciserver. You should see `(Heasoft)` at the top right of the notebook. If not, click there and select `(Heasoft)`

### Update to heasoftpy 1.2 to make sure nicerl2 works correctly

In [ ]:
import subprocess as subp
import sys, os, glob
from packaging import version
import importlib


## ------------------------------------ ##
## Make sure heasoftpy version is > 1.1 ##
## if not, grab the latest from github. ##
=p = subp.Popen(['python', '-c', 'import heasoftpy as hsp; print(hsp.__version__)'], stdout=subp.PIPE)
hsp_version = p.stdout.read().decode().strip()

if version.parse(hsp_version) < version.parse('1.2'):
    # get the latest develop version of heasoftpy and install it.
    print("Using develop version from github")
    hsplib = os.getcwd() + '/heasoftpy-develop/build/lib'
    if not os.path.exists(hsplib + '/heasoftpy'):
        os.system('wget https://github.com/HEASARC/heasoftpy/archive/refs/heads/develop.zip')
        os.system('unzip develop.zip && rm -f develop.zip')
        os.system('cd heasoftpy-develop && python setup.py build')
    sys.path.insert(0, hsplib)
## ------------------------------------ ##

import heasoftpy as hsp
print(hsp.__file__)

In [ ]:
# import other libraries
import matplotlib.pyplot as plt
import xspec
from astropy.table import Table
from astropy.time import Time
from astropy.io import fits
import astropy.units as u

# Set up the NICER obsid directory

We are using OBSID `4142010107`. The data archive is mounted under `/FTP/..`. To find the exact location of the observation, we can use `pyvo` to query the archive using the VO services, or use Xamin, as illustrated in the `Getting-Started` and `data_access` notebooks

We'll put the output from nicerl2 in an output directory separate from the input directory.

Because nicerl2 may modify of the observation directory, we copy it from the data location.

In [ ]:
nicerobsID = '4020180460'
dataLocation = f'/FTP/nicer/data/obs/2021_12/{nicerobsID}'
if not os.path.exists(nicerobsID):
    os.system(f"cp -r {dataLocation} .")
obsdir = os.path.join('.', nicerobsID)
outdir = os.path.join('.','nicerl2_output/'+nicerobsID+'_out')
# if outdir doesn't exist, create it
if not os.path.exists(outdir):
    os.makedirs(outdir)
    print(f'Created {outdir}')

# Create the nicerl2 task

In [ ]:
tstart = Time.now()
print(f'Start at: {tstart.iso[:19]}')
nicerl2 = hsp.HSPTask('nicerl2')

nicerl2.clobber = "yes"
nicerl2.indir = nicerobsID
nicerl2.cldir = outdir
nicerl2.noprompt = True

# add the KP values to the mkf file during nicerl2 processing
nicerl2.geomag_path = "/FTP/caldb/data/gen/pcf/geomag/"
nicerl2.geomag_columns = "kp_noaa.fits(KP)"

resl2 = nicerl2()

tend = Time.now()
print(f'End at: {tend.iso[:19]}')
print(f'nicerl2 took: {(tend.mjd-tstart.mjd)*86400} seconds')

if resl2.returncode != 0:
    print('\n')
    for o in resl2.output[:]:
        print(o)


# Extract products from cleaned events file

In [ ]:
clevt = f'{outdir}/ni{nicerobsID}_0mpu7_cl.evt'
phafile = f'{outdir}/ni{nicerobsID}_0mpu7_cl.pha'
lcfile = f'{outdir}/ni{nicerobsID}_0mpu7_cl.lc'
res = hsp.extractor(filename=clevt, phafile=phafile, clobber='yes', binlc=10.0,fitsbinlc=lcfile, 
                    eventsout='NONE', imgfile='NONE', regionfile='NONE', timefile='NONE', tcol='TIME',
                   ecol='PI', xcolf='RAWX', xcolh='RAWX',ycolf='RAWY', ycolh='RAWY', stokes='NONE')
print(res.stdout)

# Analyzing NICER spectra

In [ ]:
# get the on-axis rmf
res = hsp.quzcif(mission='nicer', instrument='xti',detector='-',
             filter='-', date='-', time='-',expr='-',codename='MATRIX')
rmf = [x.split()[0] for x in res.output if 'nixtiref'  in x][0]

# get the on-axis arf
res = hsp.quzcif(mission='nicer', instrument='xti',detector='-',
             filter='-', date='-', time='-',expr='-',codename='SPECRESP')
arf = [x.split()[0] for x in res.output if 'nixtiaveonaxis'  in x][0]


In [ ]:
# load the spectra (incldueing the response files) into xspec
xspec.AllData.clear()
spec = xspec.Spectrum(phafile)
spec.response = rmf
spec.response.arf = arf
spec.ignore('0.0-0.3, 10.0-**')

In [ ]:
# fit a simple absorbed broken powerlaw model
model = xspec.Model('wabs*bknpow')
xspec.Fit.perform()

In [ ]:
# plot the spectra using matplotlib

%matplotlib inline
xspec.Plot.device='/null'
xspec.Plot.xAxis='keV'
xspec.Plot('lda')
cr = xspec.Plot.y()
crerr = xspec.Plot.yErr()
en = xspec.Plot.x()
enwid = xspec.Plot.xErr()
mop = xspec.Plot.model()
target = fits.open(spec.fileName)[1].header['OBJECT']


fig = plt.figure(figsize=[8,6])
plt.ylabel('Cts/s/keV', fontsize=12)
plt.xlabel('Energy (keV)', fontsize=12)
plt.title('Target = '+target+' OBSID = '+nicerobsID+' wabs*bknpow', fontsize=12)
plt.yscale('log')
plt.xscale('log')
plt.errorbar(en, cr, xerr=enwid, yerr=crerr, fmt='k.', alpha=0.2)
plt.plot(en, mop,'r-')

# Plot the lightcurve

In [ ]:
lctab = Table.read(lcfile,hdu='RATE')
gtitab = Table.read(lcfile, hdu='GTI')
gtitab['START']=gtitab['START']-lctab.meta['TSTART']
gtitab['STOP']=gtitab['STOP']-lctab.meta['TSTART']

In [ ]:
gtitab

In [ ]:
# remove rows with no values from gtitab
row2remove=[]
for j in enumerate(gtitab):
    i=j[0]
    tsel = (lctab['TIME']>=gtitab[i]['START']) & (lctab['TIME']<=gtitab[i]['STOP'])
    if len(lctab[tsel]) < 1:
        row2remove.append(i)
gtitab.remove_rows(row2remove)
numgtis = len(gtitab)

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(1,numgtis,figsize=[10,3])
for i in range(numgtis):
    tsel = (lctab['TIME']>gtitab[i]['START']) & (lctab['TIME']<gtitab[i]['STOP'])
    t = lctab[tsel]['TIME']
    print(i, len(lctab[tsel]))
    r = lctab[tsel]['RATE']
    re = lctab[tsel]['ERROR']
    ax[i].set_ylabel('Cts/s', fontsize=12)
    ax[i].set_xlabel('Time (s)', fontsize=12)
    ax[i].set_yscale('log')
    ax[i].set_ylim(40, 500)
    ax[i].errorbar(t, r,yerr=re, fmt='k.')
    plt.tight_layout()